### Script to generate summaries using chunking based Pegasus approach

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = "IN-Abs" # Options: IN - IN-Abs, UK-UK-Abs, N2-IN-Ext
output_path = "./output/"

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
sys.path.insert(0, '../')
from utilities import *
import os
import nltk

In [ ]:
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
#Reading the test documents
names, data_source, data_summary = get_summary_data(dataset, "test")
print(len(names))
print(len(data_source))
print(len(data_summary))
len_dic = dict_names = get_req_len_dict(dataset, "test")

100
100
100
['']


In [ ]:
names


['3210.txt',
 '6270.txt',
 '362.txt',
 '3356.txt',
 '2657.txt',
 '2913.txt',
 '4963.txt',
 '5266.txt',
 '3602.txt',
 '5861.txt',
 '1181.txt',
 '2796.txt',
 '6668.txt',
 '1195.txt',
 '3292.txt',
 '3542.txt',
 '3844.txt',
 '1329.txt',
 '1778.txt',
 '6276.txt',
 '4316.txt',
 '5994.txt',
 '5141.txt',
 '4917.txt',
 '5597.txt',
 '5364.txt',
 '3531.txt',
 '4641.txt',
 '2609.txt',
 '2392.txt',
 '1697.txt',
 '2124.txt',
 '2727.txt',
 '6521.txt',
 '1974.txt',
 '652.txt',
 '6728.txt',
 '5397.txt',
 '6852.txt',
 '3893.txt',
 '6778.txt',
 '2207.txt',
 '3924.txt',
 '6118.txt',
 '5707.txt',
 '4782.txt',
 '715.txt',
 '415.txt',
 '3436.txt',
 '4568.txt',
 '6413.txt',
 '78.txt',
 '1762.txt',
 '266.txt',
 '6647.txt',
 '2035.txt',
 '6622.txt',
 '314.txt',
 '5248.txt',
 '2593.txt',
 '2649.txt',
 '2440.txt',
 '1531.txt',
 '5538.txt',
 '4807.txt',
 '2248.txt',
 '496.txt',
 '2052.txt',
 '2065.txt',
 '6881.txt',
 '3168.txt',
 '1378.txt',
 '2122.txt',
 '5937.txt',
 '380.txt',
 '4451.txt',
 '6157.txt',
 '5142.tx

In [ ]:
data_source[0]

'Appeal No. 1604 of 1967.\nFrom the judgment and order dated the 8th February, 1967 of Le Delhi High Court in Civil Writ No. 531 D of 1964.\nP.P. Rao and R. N. Sachthey, for the appellants.\nG.P. Pai, P. C. Bhartari and O. C. Mathur, for respondents nos.\n1 & 2.\nThe Judgment of the Court was delivered by SARKARIA J.\nWhether the Federation of Indian Chambers of Commerce and Industry, New Delhi (Respondent 1) is a "commercial establishment" within the meaning of s.2(5) of the Delhi Shops ,and Establishments Act, 1954 (for short the Act) is the sale question that falls for determination in this appeal by certificate directed against the judgment dated February 8, 1967 of the High Court of Delhi.\nThe facts bearing on this question may now be stated: Respondent 1 (hereinafter referred to as the Federation) is a Company registered under section 26 of the Indian Companies Act, 1913.\nThe primary objects of the Federation as given in the Memorandum of Association are "(a) To promote Indian 

In [ ]:
data_summary[0]

"The respondent was required by the Chief Inspector of Shops and Establishments to register its establishment ' under the Delhi Shops and Establishments Act, 1954.\nThe respondent contended that it was not an 'establishment ' which is defined in section 2(9) as meaning 'a shop or a commercial estab lishment ' and did not comply with the direction.\nProceedings for prosecution of its Secretary were instituted, where upon the respondent filed a petition in the High Court for quashing the order of the appellant and for directing the Magistrate not to proceed with the complaint.\nBefore the High Court the appellant contended that the respondent was a 'commercial establishment ' because, the activity of the respondent amounted to a 'Profession ', and alternatively, that the case fell within the latter part of the definition of 'commercial establishment ' in section 2(5), inasmuch as its activities were connected with trade and business generally, but, it was not urged that the activity of t

In [ ]:
device = "cuda"

In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Loading Model and tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("nsi319/legal-pegasus")
model = AutoModelForSeq2SeqLM.from_pretrained("nsi319/legal-pegasus").to(device)


In [ ]:
def summerize(text, max_len, min_len):
    '''
    Function to generate summary using Pegasus
    input:  nested_sentences - chunks
            max_l - Maximum length
            min_l - Minimum length
    output: document summary
    '''
    try:
        input_tokenized = tokenizer.encode(text, return_tensors='pt',max_length=512,truncation=True).to(device)
        summary_ids = model.generate(input_tokenized,
                                          num_beams=9,
                                          length_penalty=0.1,
                                          min_length=min_len,
                                          max_length=max_len,
                                    )
        summary = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids][0]
        return summary
    except:
        return ""

In [ ]:
def summerize_doc(nested_sentences, p):
    '''
    Function to generate summary using chunking based Pegasus
    input:  nested_sentences - chunks
            p - Number of words in summaries per word in the document
    output: document summary
    '''
    device = 'cuda'
    result = []
    for nested in nested_sentences:
        l = int(p * len(nested.split(" ")))
        max_len = l
        min_len = l-5
        result.append(summerize(nested, max_len, min_len))
    return result

In [ ]:
done_files = glob.glob(output_path + "*.txt")
done_files = [i[i.rfind("/")+1:] for i in done_files]

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# main loop to generate and save summaries of each document in the test dataset
for i in range(len(data_source)):
    done_files = glob.glob(output_path + "*.txt")
    done_files = [i[i.rfind("/")+1:] for i in done_files]
    name = names[i]
    if name in done_files:continue
    doc = data_source[i]
    input_len = len(doc.split(" "))
    req_len = dict_names[name]
    print(str(i) + ": " + name +  " - " + str(input_len) + " : " + str(req_len), end = ", ")

    nested = nest_sentences(doc,512)
    p = float(req_len/input_len)
    print(p)
    abs_summ = summerize_doc(nested,p)
    abs_summ = " ".join(abs_summ)
    print(len((abs_summ.split(" "))))

    if len(abs_summ.split(" ")) > req_len:
        abs_summ = abs_summ.split(" ")
        abs_summ = abs_summ[:req_len]
        abs_summ = " ".join(abs_summ)
    print(len((abs_summ.split(" "))))
    path = output_path + name
    file = open(path,'w')
    file.write(abs_summ)
    file.close()
#     break

0: 3210.txt - 3650 : 603, 0.1652054794520548
513
513
1: 6270.txt - 2029 : 421, 0.2074913750616067
382
382
2: 362.txt - 6285 : 1038, 0.16515513126491646
910
910
3: 3356.txt - 1874 : 354, 0.18890074706510138
284
284
4: 2657.txt - 2382 : 432, 0.181360201511335
371
371
5: 2913.txt - 1949 : 488, 0.2503848127244741
433
433
6: 4963.txt - 1286 : 596, 0.463452566096423
